In [1]:
import pandas as pd
import chime
import sys

In [2]:
df = pd.read_csv('palavras_comuns.csv')

df = df.sample(10)

df

,Palavra,Frequencia
15248,celebracao,9
12815,rogos,12
7536,morder,25
14522,acomodado,10
15610,corca,9
7592,limpo,25
8334,vencia,22
10687,aproximei,15
18311,convergir,7
18499,conspiracoes,6


#### Encontrando Sílabas

In [3]:
# Cria uma lista de vogais
vogais = ['a', 'e', 'i', 'o', 'u']

# Cria uma lista de consoantes que aceitam CA
consoantes_ca = ['b', 'c', 'd', 'f', 'g', 'p', 't']

# Cria uma lista de consoantes auxiliares (CA)
consoantes_auxiliares = ['l', 'r']

# Cria uma lista de consoantes de coda (CC)
consoantes_coda = ['m', 'n', 'r', 'l', 'z', 's', 'x']
codas = consoantes_coda

# Cria uma lista de consoantes que não aceitam CA
consoantes_nao_ca = [
    'h', 'j', 'm', 'n', 'r','l', 's', 'v', 'x', 'z', 'lh', 'nh', 'qu', 'gu', 'cl',
    'ch']

In [4]:
def gerar_silabas(vogais, consoantes_ca, consoantes_auxiliares, consoantes_coda, consoantes_nao_ca):
    silabas = []
    
    # Using only vogais
    for vogal in vogais:
        silabas.append(vogal)
        for cc in consoantes_coda:
            silabas.append(vogal + cc)
            if cc == 'n':
                silabas.append(vogal + cc + 's')
                
    # Using consoantes_ca
    for consoante in consoantes_ca:
        for vogal in vogais:
            silabas.append(consoante + vogal)
            for cc in consoantes_coda:
                silabas.append(consoante + vogal + cc)
                if cc == 'n':
                    silabas.append(consoante + vogal + cc + 's')
                    
    # Using consoantes_auxiliares
    for consoante in consoantes_auxiliares:
        for vogal in vogais:
            silabas.append(consoante + vogal)
            for cc in consoantes_coda:
                silabas.append(consoante + vogal + cc)
                if cc == 'n':
                    silabas.append(consoante + vogal + cc + 's')
                    
    # Using consoantes_nao_ca
    for consoante in consoantes_nao_ca:
        for vogal in vogais:
            silabas.append(consoante + vogal)
            for cc in consoantes_coda:
                silabas.append(consoante + vogal + cc)
                if cc == 'n':
                    silabas.append(consoante + vogal + cc + 's')
    
    return silabas

# Chama a função com os parâmetros especificados
resultado = gerar_silabas(vogais, consoantes_ca, consoantes_auxiliares, consoantes_coda, consoantes_nao_ca)

# Converte em um dataframe
silabas = pd.DataFrame(resultado, columns=['silabas'])

silabas

,silabas
0,a
1,am
2,an
3,ans
4,ar
...,...
1165,chur
1166,chul
1167,chuz
1168,chus


In [5]:
silabas.to_csv('silabas.csv')

#### 

In [6]:
def match_syllable_from_start(word, patterns, next_char_cond=None):
    """Return the longest syllable from the start of the word that matches a pattern."""
    max_syllable = ''
    for pattern in patterns:
        if word.startswith(pattern) and len(pattern) > len(max_syllable):
            # If there's a condition to check the character after the matched pattern
            next_char_index = len(pattern)
            if next_char_cond:
                # Either the word ends after the coda, or the next character satisfies the condition
                if next_char_index == len(word) or (next_char_index < len(word) and next_char_cond(word[next_char_index])):
                    max_syllable = pattern
            else:
                max_syllable = pattern
    return max_syllable

In [7]:
def identify_syllables_in_word(word, vogais, consoantes_ca, consoantes_auxiliares, consoantes_coda, consoantes_nao_ca):
    identified_syllables = []
    i = 0

    # Patterns list comprehension
    ca_vc_coda = [con + ca + v + co for con in consoantes_ca for ca in consoantes_auxiliares for v in vogais for co in consoantes_coda]
    c_v_coda = [con + v + co for con in (consoantes_ca + consoantes_nao_ca) for v in vogais for co in consoantes_coda]
    ca_v = [con + ca + v for con in consoantes_ca for ca in consoantes_auxiliares for v in vogais]
    c_v = [con + v for con in (consoantes_ca + consoantes_nao_ca) for v in vogais]
    v_coda = [v + co for v in vogais for co in consoantes_coda]
    
    while i < len(word):
        patterns = [ca_vc_coda, c_v_coda, ca_v, c_v, v_coda, vogais]
        
        matched = False
        for pattern_list in patterns:
            syllable = match_syllable_from_start(word[i:], pattern_list)
            
            # For codas, check if it should be retained as a coda or act as the starting consonant of the next syllable
            if syllable and syllable[-1] in consoantes_coda:
                if i + len(syllable) < len(word) and word[i + len(syllable)] in vogais:
                    # If coda is followed by a vowel, then treat it as the beginning of the next syllable
                    syllable = syllable[:-1]
            
            if syllable:
                identified_syllables.append(syllable)
                i += len(syllable)
                matched = True
                break

        if not matched:
            i += 1  # If no syllable match, just move to the next character

    return identified_syllables



df['Identified_Syllables'] = df['Palavra'].apply(lambda word: identify_syllables_in_word(word, vogais, consoantes_ca, consoantes_auxiliares, consoantes_coda, consoantes_nao_ca))

df

,Palavra,Frequencia,Identified_Syllables
15248,celebracao,9,"[ce, le, bra, ca, o]"
12815,rogos,12,"[ro, gos]"
7536,morder,25,"[mor, der]"
14522,acomodado,10,"[a, co, mo, da, do]"
15610,corca,9,"[cor, ca]"
7592,limpo,25,"[lim, po]"
8334,vencia,22,"[ven, ci, a]"
10687,aproximei,15,"[a, pro, xi, me, i]"
18311,convergir,7,"[con, ver, gir]"
18499,conspiracoes,6,"[con, pi, ra, co, es]"


In [8]:
chime.success()